In [5]:
# Nonlinear MPC test using kinematic bicycle model for car
# We want to use the kinematic bicycle model from this paper (Figure 3)
# https://link-springer-com.stanford.idm.oclc.org/article/10.1007/s13177-020-00226-1#Tab4

# which has states:
# x, y = positional coordinates in inertial frame
# ψ = angle of car centerline in inertial frame
# ̇ψ = change of angle
# β = angle between car centerline and velocity vector
# ̇β = change of angle

# and has the equations:
# ̇x = v cos(ψ + β)
# ̇y = v sin(ψ + β)
# ̇v = a
# ̇ψ = v / lᵣ sin(β)
# β = arctan(lᵣ/(lf + lᵣ) tan(δf))
# ̇β = v/(lf + lᵣ) tan(δf) - v/lᵣ sin(β)
# ̇δf = ω

# and we want to solve it. Define the model:


The model has state and input:
\begin{gather*}
z = \begin{bmatrix}x\\ y\\ v\\ \psi\\ \beta\end{bmatrix},\quad
u = \begin{bmatrix}a\\ \delta_f\end{bmatrix},\quad
\end{gather*}
where the two control signals are $a$, the acceleration command, and $\delta_f$, the front steering angle (in radians).
The nonlinear model is:
\begin{gather*}
\begin{bmatrix}
\dot x\\ \dot y\\ \dot v\\ \dot\psi
%\\ \dot\beta
\end{bmatrix} =
\begin{bmatrix}
v\cos(\psi + \beta)
\\
v\sin(\psi + \beta)
\\
a
\\
\frac{v}{l_r}\sin(\beta)
%\\
%\frac{v}{l_f + l_r}\tan(\delta_f) - \frac{v}{l_r}\sin(\beta)
\end{bmatrix}
\end{gather*}

## Casadi setup
From http://casadi.sourceforge.net/v3.4.4/users_guide/casadi-users_guide.pdf

In [6]:
# Library dependencies
import casadi
import numpy             as np
import matplotlib.pyplot as plt
import matplotlib        as mpl

# Local dependencies
from direct_collocation  import MpcProblem
from KinematicBicycleCar import KinematicBicycleCar
from roadrunner          import Roadrunner, RoadSegment
from road                import test_road, iso_double_lane_change

## Initial state and cost

In [7]:
# Initialize the car model with 0.05 second steps and 30 step lookahead
car = KinematicBicycleCar(N=30, step=0.025)

# Cost - we want the velocity to stay around 5 m/s, so we penalize deviations from that.
cost = (car.dae.x[0][2]-5)**2

In [8]:
n_test_road_pts,_ = np.shape(test_road)

roadrunner = Roadrunner(road_center = test_road,                    # Road centerline points
                        road_width  = np.ones(n_test_road_pts)*5.0, # constant lane width of 5 meters
                        P           = 20) # Number of points to try curve-fitting at a time

# Initial conditions: x,y = 0,0, velocity = 5.5 m/s, car at 22.5-degree angle
# This is the model:
# https://link-springer-com.stanford.idm.oclc.org/article/10.1007/s13177-020-00226-1/figures/3
ic = [test_road[0,0], test_road[0,1], 5.5, roadrunner.get_angle()]
car.set_initial(ic)

In [9]:
# Boundary functions

# terrible hack, need to fix right away
global prev_k
prev_k= 0
dist_already_traveled = 0.0

def road_center(model:KinematicBicycleCar, k:int):
    global prev_k
    if k == prev_k+1:
        roadrunner.advance(model.step*model.state_estimate[2,k])
        prev_k = k
    if k == 0:
        roadrunner.reset()
        roadrunner.advance(dist_already_traveled)
        prev_k = k
    center = np.empty((3,))
    center[0:2] = np.reshape(roadrunner.evaluate(),2)
    center[2]   = roadrunner.get_angle()
    return center

def upperbounds_x(model:KinematicBicycleCar, k:int):
    xy = road_center(model, k)
    bound = model.upperbounds_x(k)
    
    # Add the road width
    angle = roadrunner.get_angle()
    width = roadrunner.get_width()
    
    xy_0 = (width/2*np.cos(np.pi/2 + angle) + 2*np.cos(angle))
    bound[0] = xy[0] + xy_0*np.sign(xy_0)
    xy_1 = (width/2*np.sin(np.pi/2 + angle) + 2*np.sin(angle))
    bound[1] = xy[1] + xy_1*np.sign(xy_1)
    
    return bound

def lowerbounds_x(model:KinematicBicycleCar, k:int):
    xy = road_center(model, k)
    bound = model.lowerbounds_x(k)
    
    # Add the road width
    angle = roadrunner.get_angle()
    width = roadrunner.get_width()
    
    xy_0 = (width/2*np.cos(np.pi/2 + angle) + 2*np.cos(angle))
    bound[0] = xy[0] - xy_0*np.sign(xy_0)
    xy_1 = (width/2*np.sin(np.pi/2 + angle) + 2*np.sin(angle))
    bound[1] = xy[1] - xy_1*np.sign(xy_1)
    
    return bound

def upperbounds_u(model:KinematicBicycleCar, k:int):
    bound = model.upperbounds_u(k)
    return bound

def lowerbounds_u(model:KinematicBicycleCar, k:int):
    bound = model.lowerbounds_u(k)
    return bound

In [10]:
# What we should do now
# 1. Use the velocity estimate to "step through" the road,
#    obtaining a set of road midpoints and constraint boundaries
# 2. Add a term to the cost function that attracts the successive states to the road midpoints - done
# 3. Use the midpoints to initialize the variable estimate so the solver is faster. - done,
#    and we are initializing the new control uk to the previously computed one
#    kind of warm-starting ipopt

#from collections import deque

#v_estimate = np.ones(car.N)*ic[2]
#road_points = deque(maxlen=car.N)
#dist = 0.0
#roadrunner.reset()

#for v in v_estimate:
#    roadrunner.advance(v*car.step)
#    xy = roadrunner.evaluate()
#    road_points.append(np.reshape(xy,2))


## Run the MPC algorithm

In [ ]:
roadrunner.reset()
prev_k = 0

mpcprob = MpcProblem(car,     # casadi.DaeBuilder()
                     cost,        # casadi symbolic objective
                     upperbounds_x=upperbounds_x,
                     lowerbounds_x=lowerbounds_x,
                     upperbounds_u=upperbounds_u,
                     lowerbounds_u=lowerbounds_u,
                     road_center=road_center,
                    )

# Controls the NUMBER of iterations to run the MPC algorithm.
n_runs = 250

x_plan = np.empty((mpcprob.model.n,n_runs+1))
u_plan = np.empty((mpcprob.model.m,n_runs))
x_true = np.empty((mpcprob.model.n,n_runs+1))
x_true[:,0] = ic
x_plan[:,0] = ic

boundary_up  = np.empty((n_runs,2))
boundary_low = np.empty((n_runs,2))


for k in range(n_runs):
    xk_opt, uk_opt = mpcprob.run(x_true[:,k])
    for i in range(mpcprob.model.n):
        x_plan[i,k+1] = xk_opt[i][1]
    for i in range(mpcprob.model.m):
        u_plan[i,k] = uk_opt[i][1]
    x_true[:,k+1] = mpcprob.simulate(x_true[:,k], u_plan[:,k])
    
    dist_already_traveled += np.sqrt((x_true[0,k+1]-x_true[0,k])**2 + (x_true[1,k+1]-x_true[1,k])**2)

    boundary_up[k,:] = upperbounds_x(mpcprob.model, 0)[0:2]
    boundary_low[k,:] = lowerbounds_x(mpcprob.model, 0)[0:2]
        
    
    # hack to demonstrate changing the cost halfway through the number of runs
    #if k = n_runs//2:
        # now the cost is penalizing velocity, so the car will stop.
    #    mpcprob.set_cost(100*car.dae.x[0][2]**2)

# This was to compare the error between the prediction and
# the simulation using casadi ODE integrator
# the error is very small.
#for k in range(1,n_runs):
    #print("True:", x_true[:,k])
    #print("Pred:", x_plan[:,k])
    #print("Error:", np.linalg.norm(x_true[:,k]-x_plan[:,k],2))

# Get the planned boundaries
boundary_plan_up = np.empty((mpcprob.model.N,2))
boundary_plan_low = np.empty((mpcprob.model.N,2))
for k in range(mpcprob.model.N):
    boundary_plan_up[k,:] = upperbounds_x(mpcprob.model, k)[0:2]
    boundary_plan_low[k,:] = lowerbounds_x(mpcprob.model, k)[0:2]
    
# Add them to the boundaries for past states
boundary_up = np.concatenate([boundary_up, boundary_plan_up])
boundary_low = np.concatenate([boundary_low, boundary_plan_low])

Attracting  X_1[0] 1 to  [0.59843593 0.80269419 0.06659183]
Attracting  X_2[0] 2 to  [0.71297965 0.81051745 0.06659183]
Attracting  X_3[0] 3 to  [0.82762372 0.8184649  0.06659183]
Attracting  X_4[0] 4 to  [0.94236138 0.82653164 0.06659183]
Attracting  X_5[0] 5 to  [1.05718645 0.83471285 0.06659183]
Attracting  X_6[0] 6 to  [1.17209337 0.84300371 0.06659183]
Attracting  X_7[0] 7 to  [1.28707714 0.85139947 0.06659183]
Attracting  X_8[0] 8 to  [1.40213326 0.85989543 0.06659183]
Attracting  X_9[0] 9 to  [1.51725774 0.86848693 0.06659183]
Attracting  X_10[0] 10 to  [1.63244706 0.87716938 0.06659183]
Attracting  X_11[0] 11 to  [1.74769813 0.88593828 0.06659183]
Attracting  X_12[0] 12 to  [1.86300828 0.89478917 0.06659183]
Attracting  X_13[0] 13 to  [1.97837521 0.90371767 0.06659183]
Attracting  X_14[0] 14 to  [2.09379698 0.91271948 0.06659183]
Attracting  X_15[0] 15 to  [2.20927202 0.92179038 0.06659183]
Attracting  X_16[0] 16 to  [2.32479904 0.93092622 0.06659183]
Attracting  X_17[0] 17 to 

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2834
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      480

Total number of variables............................:      540
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      540
                     variables with only upper bounds:        0
Total number of equality constraints.................:      480
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2834
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      480

Total number of variables............................:      540
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      540
                     variables with only upper bounds:        0
Total number of equality constraints.................:      480
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

   8  1.3518737e-04 4.36e-08 1.48e-12  -5.7 5.19e-01    -  1.00e+00 1.00e+00h  1
   9  1.3072835e-04 5.77e-10 1.12e-07  -8.6 1.91e-01    -  9.63e-01 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.3021923e-04 1.59e-11 1.43e-15  -8.6 2.88e-02    -  1.00e+00 1.00e+00h  1
  11  1.3020971e-04 4.65e-11 4.84e-12  -9.0 5.13e-04    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 11

                                   (scaled)                 (unscaled)
Objective...............:   1.3020971027208209e-04    1.3020971027208209e-04
Dual infeasibility......:   4.8395701046017481e-12    4.8395701046017481e-12
Constraint violation....:   4.6500150951978014e-11    4.6500150951978014e-11
Complementarity.........:   9.1127068758993708e-10    9.1127068758993708e-10
Overall NLP error.......:   9.1127068758993708e-10    9.1127068758993708e-10


Number of objective function evaluations             = 12
Number of objective gradient evaluations        

   5  3.4940682e-04 3.96e-08 6.43e-05  -5.7 8.16e-01  -4.0 9.23e-01 1.00e+00h  1
   6  1.7420600e-04 2.81e-08 4.70e-06  -5.7 1.65e+00    -  8.63e-01 1.00e+00h  1
   7  1.4419178e-04 2.47e-08 1.19e-06  -5.7 7.03e-01    -  9.40e-01 1.00e+00h  1
   8  1.3287454e-04 1.79e-09 3.50e-07  -8.6 4.62e-01    -  9.53e-01 1.00e+00h  1
   9  1.3038725e-04 2.23e-10 7.42e-13  -8.6 1.24e-01    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.3020341e-04 3.47e-09 3.04e-11  -8.6 9.20e-03    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 10

                                   (scaled)                 (unscaled)
Objective...............:   1.3020341153649524e-04    1.3020341153649524e-04
Dual infeasibility......:   3.0385358038324990e-11    3.0385358038324990e-11
Constraint violation....:   3.4747154181724582e-09    3.4747154181724582e-09
Complementarity.........:   3.4246499400475131e-09    3.4246499400475131e-09
Overall NLP error......

...

Attracting  X_3[0] 3 to  [25.37074924  1.88025634 -0.0939236 ]
Attracting  X_4[0] 4 to  [25.48880965  1.86926992 -0.0939236 ]
Attracting  X_5[0] 5 to  [25.6070179   1.85830127 -0.0939236 ]
Attracting  X_6[0] 6 to  [25.7253713   1.84735364 -0.0939236 ]
Attracting  X_7[0] 7 to  [25.84386719  1.83643018 -0.0939236 ]
Attracting  X_8[0] 8 to  [25.9625029   1.82553395 -0.0939236 ]
Attracting  X_9[0] 9 to  [26.08127579  1.8146679  -0.0939236 ]
Attracting  X_10[0] 10 to  [26.20018325  1.80383491 -0.0939236 ]
Attracting  X_11[0] 11 to  [26.31922271  1.79303772 -0.0939236 ]
Attracting  X_12[0] 12 to  [26.43839162  1.782279   -0.0939236 ]
Attracting  X_13[0] 13 to  [26.55768752  1.7715613  -0.0939236 ]
Attracting  X_14[0] 14 to  [26.67710794  1.76088704 -0.0939236 ]
Attracting  X_15[0] 15 to  [26.79665051  1.75025858 -0.0939236 ]
Attracting  X_16[0] 16 to  [26.9163129   1.73967812 -0.0939236 ]
Attracting  X_17[0] 17 to  [27.03609284  1.72914777 -0.0939236 ]
Attracting  X_18[0] 18 to  [27.15598813

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.3020639e-04 1.45e-08 4.29e-10  -8.6 2.53e-03    -  1.00e+00 1.00e+00h  1
  11  1.3020477e-04 6.97e-14 2.08e-09  -9.0 6.24e-05  -4.5 1.00e+00 1.00e+00h  1

Number of Iterations....: 11

                                   (scaled)                 (unscaled)
Objective...............:   1.3020476781758884e-04    1.3020476781758884e-04
Dual infeasibility......:   2.0810729149356234e-09    2.0810729149356234e-09
Constraint violation....:   6.9722005946459831e-14    6.9722005946459831e-14
Complementarity.........:   9.0939459847406065e-10    9.0939459847406065e-10
Overall NLP error.......:   2.0810729149356234e-09    2.0810729149356234e-09


Number of objective function evaluations             = 12
Number of objective gradient evaluations             = 12
Number of equality constraint evaluations            = 12
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobia

  11  1.4939026e-04 6.31e-05 6.23e-06  -5.7 2.00e+00    -  1.00e+00 5.00e-01f  2
  12  1.3708425e-04 2.22e-05 4.28e-09  -5.7 3.19e-01    -  1.00e+00 1.00e+00h  1
  13  1.3246198e-04 1.24e-07 7.97e-12  -5.7 1.95e-01    -  1.00e+00 1.00e+00h  1
  14  1.3030969e-04 4.19e-09 4.38e-08  -8.6 1.00e-01    -  9.81e-01 1.00e+00h  1
  15  1.3018036e-04 2.79e-08 3.97e-09  -8.6 8.59e-03    -  1.00e+00 1.00e+00h  1
  16  1.3017998e-04 9.31e-14 1.82e-09  -8.6 1.48e-05  -3.9 1.00e+00 1.00e+00h  1

Number of Iterations....: 16

                                   (scaled)                 (unscaled)
Objective...............:   1.3017997728449773e-04    1.3017997728449773e-04
Dual infeasibility......:   1.8216942038225154e-09    1.8216942038225154e-09
Constraint violation....:   9.3050567251395933e-14    9.3050567251395933e-14
Complementarity.........:   2.5059457630891128e-09    2.5059457630891128e-09
Overall NLP error.......:   2.5059457630891128e-09    2.5059457630891128e-09


Number of objective funct

  13  1.3335991e-04 1.36e-04 1.62e-08  -5.7 1.15e-01    -  1.00e+00 1.00e+00f  1
  14  1.3207393e-04 1.02e-06 1.50e-10  -5.7 5.74e-02    -  1.00e+00 1.00e+00h  1
  15  1.3031526e-04 2.71e-09 3.57e-08  -8.6 8.33e-02    -  9.85e-01 1.00e+00h  1
  16  1.3022839e-04 5.39e-12 2.48e-13  -8.6 6.16e-03    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 16

                                   (scaled)                 (unscaled)
Objective...............:   1.3022839123920870e-04    1.3022839123920870e-04
Dual infeasibility......:   2.4793063901269412e-13    2.4793063901269412e-13
Constraint violation....:   5.3911614755763893e-12    5.3911614755763893e-12
Complementarity.........:   2.7113672991997485e-09    2.7113672991997485e-09
Overall NLP error.......:   2.7113672991997485e-09    2.7113672991997485e-09


Number of objective function evaluations             = 19
Number of objective gradient evaluations             = 17
Number of equality constraint evaluations            = 19
Number of ine

  11  1.3223054e-04 6.96e-05 7.76e-09  -5.7 1.38e-01    -  1.00e+00 1.00e+00h  1
  12  1.3198270e-04 1.92e-07 1.13e-11  -5.7 1.11e-02    -  1.00e+00 1.00e+00h  1
  13  1.3025967e-04 2.01e-09 3.50e-08  -8.6 8.18e-02    -  9.85e-01 1.00e+00h  1
  14  1.3017616e-04 1.60e-12 2.24e-13  -8.6 5.97e-03    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 14

                                   (scaled)                 (unscaled)
Objective...............:   1.3017615678331324e-04    1.3017615678331324e-04
Dual infeasibility......:   2.2361672179867526e-13    2.2361672179867526e-13
Constraint violation....:   1.5964018301728444e-12    1.5964018301728444e-12
Complementarity.........:   2.7017828857914229e-09    2.7017828857914229e-09
Overall NLP error.......:   2.7017828857914229e-09    2.7017828857914229e-09


Number of objective function evaluations             = 22
Number of objective gradient evaluations             = 15
Number of equality constraint evaluations            = 22
Number of ine

## Control signals: acceleration and steering angle

In [ ]:
u_plt, u_ax1, u_ax2 = car.plot_u(u_plan, mpcprob.u_opt)
# control limits

## x-y plot of car's path
Arrows show velocity. Orange components are planned but not executed.

In [ ]:
x_plt, x_ax = car.plot_x(x_plan, mpcprob.x_opt)
for i in range(0,5):
    roadrunner.segment_ptr = i
    x = np.linspace(0,1,20)
    xy = roadrunner.evaluate(x)
    #xy = roadrunner.to_world_frame(xy, roadrunner.angle[i], roadrunner.road_center[i,:])
    x_ax.scatter(xy[:,0], xy[:,1])
    
x_ax.scatter(test_road[0:25,0], test_road[0:25,1], color='lightgreen')

plt.show()

## x vs time and y vs time, separately, with the boundaries

In [ ]:
print("Final state:")
print("(x,y):", x_plan[0,-1], x_plan[1,-1])

fig1, ax1, ax2 = mpcprob.model.plot_with_time(x_plan, mpcprob.x_opt, boundary_up, boundary_low)